# (Pred)spracovanie a analýza údajov

## Rozumieť údajom

- predpracovanie je jedna z najdôležitejších častí

- iba keď máte o vašich dátach predstavu, budete ich vedieť správne spracovať a vyhodnotiť 

### 6 fáz:

- Objavovanie (explore)
- Očistenie (clean)
- Úpravy (mutate)
- Popis (describe)
- Vizualizácia (vizualize)
- Analýza (analyze)

In [ ]:
# R
# install and load necessary libraries

install.packages("tidyverse")
install.packages("ggplot2")
install.packages("data.table")
install.packages("corrplot")

library(tidyverse)

## Objavovanie (explore)

Budeme používať dataset U.S. Chronic Disease Indicators (CDI), dostupný na:
- https://catalog.data.gov/dataset/u-s-chronic-disease-indicators-cdi

1. načítanie datasetu - aby sme nemuseli dataset manuálne sťahovať, použijeme vstavané funkcie R pre stiahnutie a načítanie súboru

In [ ]:
# URL to download the dataset
dataset_url <- "https://data.cdc.gov/api/views/g4ie-h725/rows.csv?accessType=DOWNLOAD"

# destination file path
dest_file <- "" # TODO - fill in

# download the dataset
download.file(dataset_url, destfile = dest_file)

# read the CSV file into R
data <- read.csv(dest_file)

2. Zobrazenie základých informácií o datasete

   (Pri dlhých výstupoch je dobré zapnúť posuvný mód - klik pravým tlačidlom myši na Output a výber 'Enable Scrolling for Outputs')

In [ ]:
# glimpse shows variables, variable type and first values
glimpse(data)

In [ ]:
# view - similar to glimpse, but formatted
# !! doesn't display all the variables, if there are too many
View(data)

3. zobrazenie sumarizácie pre všetky premenné

   - obsahuje základné matematické štatistiky pre numerické premenné
   - obsahuje typ kategorickej premennej
  
   + **jednoduché zistenie zbytočných premenných**

In [ ]:
summary(data)

4. Zobrazenie prvých a posledných riadkov datasetu

   - overenie, či bol dataset načítaný správne

In [ ]:
head(data)
tail(data)

5. Zobrazenie názvov premenných

    - užitočné, pri vytváraní tabuliek, grafov, ..

In [ ]:
colnames(data)

##### A. Kategorické premenné

6. Overenie hodnôt v premenných

In [ ]:
# there are only NA's
unique(data$Response)
unique(data$StratificationID2)

In [ ]:
# there are more values
unique(data$Topic)

7. Vytvorenie tabuľky

   - počet záznamov v datasete pre danú hodnotu premennej
   - tabuľku môžete tiež vytvárať pre viac premenných (premenné sú oddelené čiarkou), výsledkom je tabuľka, ktorá obsahuje počet záznamov vyskytujúcich sa v datasete pre jednotlivé kombinácie hodnôt vybraných premenných 

In [ ]:
table(data$LocationDesc)

8. Triedenie tabuľky

    - predvolene triedená podľa hodnôt premennej podľa abecedy
    - utrieďte tabuľku tak, aby na začiatku tabuľky boli štáty ktoré majú v datasete najviac záznamov  
    - funkcia sort() - www.rdocumentation.org/packages/base/versions/3.6.2/topics/sort


In [ ]:
# TODO - sort the table

9. Grafické zobrazenie počtu hodnôt

    - vytvorte čiarový graf, ktorý vám zobrazí počet záznamov pre jednotlivé kategórie (premenná Topics) v datasete  
    - funkcia barplot() - www.rdocumentation.org/packages/graphics/versions/3.6.2/topics/barplot
    - ďalšie formátovanie - www.r-graph-gallery.com/210-custom-barplot-layout.html  

In [ ]:
# settings for plot visualisation
options(repr.plot.height = 5, repr.plot.res = 200)

# TODO - create barplot from the previously sorted table (default = vertical)

# TODO - set the labels' orientation to perpendicular (find 'las' attribute in the advanced formatting link)

# increase margin size to fit
# par(mar=c(20,5,1,1)) # vertical

# TODO - change the barplot to horizontal (find 'horiz' attribute in the advanced formatting link)

# par(mar=c(5,20,1,1)) # horizontal


   
##### B. Numerické premenné

10. Zobrazenie matematických vlastností
    
    - variancia (variance): priemerná kvadratická odchýlka od priemeru
    - smerodajná odchýlka (standard deviation): druhá odmocnina variancie
    - horný/dolný kvartil (quartile): hodnota, od ktorej je 25%, resp. 75% hodnôt menších
    - medián (median) - rozdeľuje súbor údajov na dve rovnaké polovice (polovica údajov je pred ním, polovica za ním)
    - priemer (mean) - súčet všetkých hodnôt delený počtom hodnôt, ovplyvnený extrémnymi hodnotami
    - NA - prázdne hodnoty

Možnosť použiť ďalšie funkcie ako sd(), var(), boxplot.stats() a pod.

In [ ]:
# summary shows min, max, 1.st and 3.rd Quartile, Median, Mean and NA's
summary(data$DataValueAlt)

11. Zobrazenie chýbajúcich hodnôt v datasete
    - chýbajúce hodnoty - NA - www.rdocumentation.org/packages/base/versions/3.6.2/topics/NA

In [ ]:
# see data in variable
data$LowConfidenceLimit

# is.na - function that returns logical value, whether the value is NA or not
is.na(data$LowConfidenceLimit)

In [ ]:
# display all the variables of all the data, where LowConfidenceLimit is NA
data[is.na(data$LowConfidenceLimit),] 

## Očistenie (clean)

Očistenie údajov zahŕňa: 
- overenie/úpravu typov premenných
- výber podstatných premenných
- filtrovanie hodnôt, podľa toho čo chceme ďalej spracovávať
- odstránenie NA a duplicitných údajov

1. Overenie a zmena typu premennej
    - pri kategorických premenných je dobré zmeniť typ premennej z 'character' na 'factor', pretože pri niektorých štatistických testoch (napr. ANOVA) sa môže stať, že  táto premenná nebude vyhodnocovaná ako kateogrická, čo ovplyvní typ a výsledok použitého testu
  
    - as.factor - www.rdocumentation.org/packages/h2o/versions/3.42.0.2/topics/as.factor


In [ ]:
# class function shows the variable type
class(data$Topic)

In [ ]:
# change categorical variable types to factor
data$LocationAbbr <- as.factor(data$LocationAbbr)
data$LocationDesc <- as.factor(data$LocationDesc)
data$DataSource <- as.factor(data$DataSource)
data$Topic <- as.factor(data$Topic)
data$TopicID <- as.factor(data$TopicID)
data$Question <- as.factor(data$Question)
data$QuestionID <- as.factor(data$QuestionID)
data$StratificationCategory1 <- as.factor(data$StratificationCategory1)
data$DataValueType <- as.factor(data$DataValueType)

2. Overenie hodnôt premennej (špecifické pre premenné typu factor)

   - rozdiel unique/factor - unique vracia unikátne hodnoty prítomné vo vzorke, factor vracia všetky hodnoty, ktoré má premenná definované a môže ich nadobúdať (niečo ako výber hodnoty z dropdownu)

In [ ]:
levels(data$StratificationCategory1)

3. Výber podstatných premenných

   - redukcia datasetu - len premenné ktoré nás zaujímajú/majú pre nás nejakú hodnotu
   - funkcia select - https://dplyr.tidyverse.org/reference/select.html
   - pre opakujúce sa názvy použite funkciu starts_with - https://www.rdocumentation.org/packages/tidyselect/versions/1.2.0/topics/starts_with

**Ktoré premenné na základe doterajších znalostí môžeme vynechať?**

In [ ]:
# %>% in R means redirecting the output to some function, similar to >> in bash
 #TODO

V premennej DataValueUnit sa v niektorých záznamoch nachádzajú prázdne hodnoty. 

**Malo by význam ich vyfiltrovať ?** 

Ak áno tak kedy ?

4. Výber záznamov s ktorými chceme pracovať

   - napríklad chceme pracovať len s otázkami ktoré sa týkajú cukrovky (TopicID = 'DIA') a boli zaznamenané od roku 2015.
   - doplňte predošlý výber údajov o filtrovanie tak, že výstup z funkcie select presmerujete do ďalšej funkcie, kde vyfiltrujete údaje
   - použite funkciu filter - https://dplyr.tidyverse.org/articles/grouping.html?q=filter#filter
   - pre porovnávanie faktorov/stringov môžete použiť %in% operator - NázovPremennej %in% "HodnotaPremennej"
   - operátory v R - https://www.statmethods.net/management/operators.html
   - priraďte output do premennej `data_chosen`

In [ ]:
# TODO
# use function filter(condition1 & condition2)

5. Odstránenie chýbajúcich hodnôt

    - dôležité najmä pre numerické premenné - použitie matematických funkcií 

In [ ]:
# numeric variable with non-numeric datatype
mean(data_chosen$DataValue)

In [ ]:
# numeric variable with numeric datatype but also NA's
mean(data_chosen$DataValueAlt)

In [ ]:
# numeric variable with numeric datatype but also NA's
mean(data_chosen$DataValueAlt, na.rm = TRUE)

6. Zobrazenie nekompletných záznamov

In [ ]:
# the dot passes the input of the filter function as an input to the complete.cases function
data_chosen %>% 
    filter(!complete.cases(.))

Vymazanie záznamov, ktoré majú v stĺpci DataValueAlt hodnotu NA

In [ ]:
data_filtered <- data_chosen %>% 
    drop_na(DataValueAlt)

data_filtered

Overenie, či dataset obsahuje ešte nejaké NA hodnoty:

In [ ]:
# TODO

7. Odstránenie duplicitných hodnôt
    
    - funkcia distinct - https://www.rdocumentation.org/packages/dplyr/versions/1.0.10/topics/distinct

In [ ]:
data_clean <- data_filtered %>% 
    distinct()

## Úpravy (mutate)

- ak z nejakých dôvodov potrebujeme urobiť zmeny premenných
    - prekódovanie premennej
    - zmena názvu premennej
    - vytvorenie pivotovej tabuľky a pod.

1. Prekódovanie hodnôt

   - napr. ak chceme niektorú z kategorických premenných označiť číselným kódovaním, prípadne zmeniť ich poradie, atď.

In [ ]:
unique(data_clean$Stratification1)
unique(data_clean$StratificationID1)

Aký dátový typ má premenná `Stratification1` ?

In [ ]:
# TODO

Aky typ musí mať aby bola kategorická?

Zmeňte typ premennej `StratificationID1` na kategorický a nastavte poradie hodnôt v tejto premennej.

- Použite funkciu factor - https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/factor

In [ ]:
# TODO
# variable <- factor(variable, levels = c( ... (values) )

Vytvorte v datasete `data_clean` novú premennú s názvom `Stratification1ID_code`, ktorá bude obsahovať zakódované hodnoty premennej  `Stratification1ID` vo forme celých čísel.
- Použite funkciu as.integer - https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/integer

In [ ]:
# TODO

2. Zmena názvu premennej
- funkcia rename - https://dplyr.tidyverse.org/reference/rename.html

In [ ]:
data_clean <- data_clean %>%
    rename("DataValueNum" = "DataValueAlt")  # target = source

3. Vytvorenie kontingenčnej tabuľky (pivot table)

   - vzájomný vzťah dvoch premenných
   - stĺpce ukazujú hodnoty jednej premennej, riadky druhej premennej
   - hodnoty v tabuľke ukazujú agregované hodnoty - počet, maximum, minimum, priemer, ..
  
Na vytvorenie sa môže použiť funkcia pivot_wider alebo pivot_longer

In [ ]:
# select specific data for pivot table only
data_pivot <- select(data_clean, YearStart, LocationAbbr, DataValueNum) 

# pivot table in wider format where columns are YearStart values, 
# rows are LocationAbbr and values are the mean of DataValueAlt
pivot_table <- data_pivot %>%
    pivot_wider(names_from = YearStart, 
                values_from = DataValueNum, 
                values_fn = list(DataValueNum = mean))

pivot_table

## Popis (describe)

- popis intervalu, v ktorom sa údaje nachádzajú
- ich sústredenie (centralizovanie) okolo stredu
- rozptyl
- sumarizácia
- vytvorenie tabuliek

1. Popis matematických vlastností

In [ ]:
min(data_clean$DataValueNum)
max(data_clean$DataValueNum)

# min - max
range(data_clean$DataValueNum)

# interquartile range
IQR(data_clean$DataValueNum)

mean(data_clean$DataValueNum, na.rm = TRUE)
median(data_clean$DataValueNum)

var(data_clean$DataValueNum)

2. Vytvorenie tabuľky s použitím sumarizácie

   - zoskupovanie údajov - funkcia group_by - https://dplyr.tidyverse.org/reference/group_by.html
   - funkcia summarise - https://dplyr.tidyverse.org/reference/summarise.html

In [ ]:
data_clean %>%
    group_by(LocationAbbr) %>%
    summarise(MinValue = min(DataValueNum), 
              MaxValue = max(DataValueNum), 
              Mean = mean(DataValueNum),
              SD = sd(DataValueNum))

## Vizualizácia (vizualize)

- grafy pre jednoduché pochopenie údajov
- oddelenie kategorickej závislej premennej v priestore nezávislých premenných
    - napr. farebné oddelenie chorých/zdravých pacientov v grafe priemerných hodnôt krvného tlaku a veku
 
+ najpoužívanejšia knižnica - ggplot2 - https://ggplot2.tidyverse.org/

1. Stĺpcový graf Výskytu cukrovky podľa lokality

In [ ]:
ggplot(data_clean, aes(x = LocationAbbr, y = DataValueNum)) +
  geom_col() +
  theme(axis.text.x = element_text(angle = 90)) +
  labs(title = "Prevalence of Diabetes by Location", y = "Prevalence (%)", x = "Location")

6. Boxplot diagram

   - čiara - medián
   - box - medzikvartilové rozpätie - obsahuje 50% hodnôt
   - hranice - minimum, maximum
  
<img src="data/boxplot.png " alt="Workflow diagram" width="70%"/>

Pozorovania:

- Medián zaočkovanosti je vo všetkých skupinách v intervale 40-60 %.
- Medzikvartilové rozpätie (IQR - box) je vo väčšine skupín podobné, čo naznačuje konzistentné rozloženie údajov okolo mediánu.
- Najmenšia vaeriabilita je v skupine "Multiracial, non-Hispanic".
- V niekoľkých skupinách sa vyskytujú odľahlé hodnoty, najmä v skupinách "Black, non-Hispanic, Male a Overall", čo poukazuje na niektoré osoby s výrazne nižšou mierou zaočkovanosti.

## Analýza (analyze)


- Odhalenie závislostí (napr. korelácia)
- Štatistické testy
- Algoritmy (dimenzionálna redukcia, strojové učenie, ...)

Dataset Breast Cancer Wisconsin (Original)

- dostupný na https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original

- premenné:
    - Sample_code_number (ID),
    - Clump_thickness - hrúbka zhluku
    - Uniformity_of_cell_size - Uniformita veľkosti buniek
    - Uniformity_of_cell_shape - uniformita tvaru buniek
    - Marginal_adhesion - Priľnavosť okrajov
    - Single_epithelial_cell_size - veľkosť jednej epitelovej bunky
    - Bare_nuclei - "holé" jadrá (bez cytoplazmy)
    - Bland_chromatin - nevýrazný chromatín
    - Normal_nucleoli - normálne jadrá
    - Mitoses - mitózy
    - Class - typ nádoru (4 Malígny, 2 Benígny)

In [ ]:
# load the dataset
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"

# change the variable names
data_bc <- read.csv(file = url, header = FALSE,
                col.names = c("id","CT", "UCSize", "UCShape", "MA", "SECS", "BN", "BC", "NN","M", "diagnosis"))
head(data_bc)

# mutate the diagnosis variable
data_bc$diagnosis[data_bc$diagnosis==4] = 1 # malign
data_bc$diagnosis[data_bc$diagnosis==2] = 0 # benign

#filter to check and review any duplicate records
data_bc <- data_bc %>% 
    distinct()

# select only relevant variables
data_bc <- data_bc %>% select(-id, -BN)

head(data_bc)


1. Vzťahy medzi premennými

   - premenné sa môžu navzájom ovplyvňovať
   - lineárne alebo nelineárne vzťahy
   - vizualizácia pre jednoduché pochopenie, či je možné pomocou vybraných premenných jednoznačne oddeliť hodnoty závislej premennej (v tomto prípade benígne a malígne nádory)

In [ ]:
ggplot(data_bc, aes(x = M, y = BC, color = diagnosis)) +
  geom_point() +
  ggtitle("Scatter Plot of M vs BC")

In [ ]:
ggplot(data_bc, aes(x = UCSize, y = UCShape, color = diagnosis)) +
  geom_point() +
  ggtitle("Scatter Plot of UCSize vs UCShape")

2. Korelácia premenných

    - V intervale <-1, 1> 
    - pozitívna (1) - so zvyšovaním jednej premennej sa zvyšuje aj druhá
    - negatívna (-1) - so zvyšovaním jednej premennej sa druhá premenná znižuje
    - nulová (0) - premenné sa neovplyvňujú


   Korelácia neznamená kauzalitu - dva javy sa môžu ukazovať ako závislé, ale neznamená to, že jeden spôsobuje vznik druhého.

In [ ]:
library(corrplot)

# create a correlation matrix
data_bc_matrix <- cor(data_bc)

# create a correlation plot
corrplot(data_bc_matrix, order="hclust", type='upper',tl.srt = 45, tl.cex = 0.6)